In [261]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [262]:
product_type = "frozen"
num_time_periods = 7

In [263]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [264]:

if(product_type=="frozen"):
    warehouse_volume = 30
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [265]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [266]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [267]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [268]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [269]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [270]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [271]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

In [272]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand- item_demand[t], disposal_cost)
                tot_ordering += math.ceil((cum_demand- item_demand[t])/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


70982.59999999999
Ordering 60495.79999999998
Fixing  7953.5999999999985
Clearance  2526.9000000000015
Max Order  6.300000000000001


In [273]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [274]:
cost_dict['total'] = obj_val
cost_dict['buffer'] = buf_cost
cost_dict['clearance'] = tot_disposal
cost_dict['fixed'] = tot_fixed
cost_dict['per_tu'] = tot_ordering
cost_dict['max_order_penalty'] = tot_max_order
cost_dict

{'fixed': 7953.5999999999985,
 'per_tu': 60495.79999999998,
 'clearance': 2526.9000000000015,
 'buffer': 149.81307894999975,
 'max_order_penalty': 6.300000000000001,
 'total': 71132.41307894999}

In [275]:
obj_val

71132.41307894999

In [276]:
schedule

{'00ee8964': {0: 32, 5: 14},
 '010c48db': {0: 1, 1: 6},
 '015eb098': {0: 2, 3: 2},
 '019f5beb': {0: 4},
 '020b50b0': {0: 6, 3: 5},
 '03436da5': {0: 7, 4: 4},
 '038bb02c': {0: 1, 2: 1, 5: 1},
 '04112e43': {0: 1, 3: 3},
 '04828ea6': {0: 7, 3: 4, 4: 10},
 '050daac5': {0: 10},
 '05fa4114': {0: 7, 2: 12, 5: 5},
 '05ffd974': {0: 14, 4: 6},
 '0744ee79': {0: 2, 3: 2},
 '0793a340': {0: 1, 3: 1},
 '097bbc43': {0: 11, 2: 20},
 '0983d839': {0: 5, 5: 2},
 '09e397f8': {0: 17, 4: 8},
 '0b1a45e0': {0: 1, 2: 1, 4: 1},
 '0ba6590b': {0: 5, 1: 44, 5: 12},
 '0e611aba': {0: 9},
 '1017897f': {0: 4, 1: 11, 3: 18},
 '105e43d9': {0: 7},
 '106ac8f0': {0: 5, 1: 10, 3: 16},
 '10f2b453': {0: 4, 2: 9},
 '111b2434': {0: 1, 2: 5},
 '11545bca': {0: 11, 3: 9},
 '126334a5': {0: 16, 3: 15},
 '12c460b7': {0: 4, 1: 20},
 '12cf1bca': {0: 85, 4: 37},
 '13541e66': {0: 11},
 '1377878b': {0: 159},
 '13becd5f': {0: 3, 3: 4},
 '148ed136': {0: 4, 4: 3},
 '15b0e7a5': {0: 12},
 '17f9fecb': {0: 5, 3: 3},
 '189067f1': {0: 17, 4: 9},
 '

In [277]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = math.inf
    cost = obj_val
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule 
    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            cost_improvement = 0
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i + 1
                    if order_days[day2] - order_days[day1] <= shelf_life[item]:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            cost += delta
                            modified_orders[order_days[day1]] = tot_orders
                            volumes[order_days[day1]] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                modified_schedule[item] = modified_orders
            
            else:
                if len(schedule[item]) == 1 : 
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)

        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement
    
    return schedule, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [278]:
schedule, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 0
71132.41307894999
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 0
71132.41307894999
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 0
71132.41307894999
Buffer penalty paid 0
max order penalty paid 0
fixed cost reduced by 0
71132.41307894999


{'fixed': 7953.5999999999985,
 'per_tu': 60495.79999999998,
 'clearance': 2526.9000000000015,
 'buffer': 149.81307894999975,
 'max_order_penalty': 6.300000000000001,
 'total': 71132.41307894999}

In [279]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

In [280]:
new_clearance_cost = 0
ordering_cost_per_tu_saved = 0

# Add clearance cost
# If I ever have to throw away equal to or more than a TU's worth, then I can just reduce order
def removeExpired(inv,life,item):
    global ordering_cost_per_tu_saved
    global new_clearance_cost
    
    tu_saved = 0
    # if inventory has items from days which are now too old
    if(len(inv)>life):
        amount_cleared = inv[0]

        tu_saved = amount_cleared//cu_per_tu[item]
        # ordering cost reduced here
        ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

        amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
        # clearance cost recalculated here
        new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]
        
        # remove days which are now too old
        inv = inv[-life:]
    
    return inv, tu_saved

def addDay(inv,ordered):
    inv.append(ordered)
    return inv

# how much of the current demand can be satisfied from items in inventory
def satisfyFromInventory(inv, dem):
    for day in range(len(inv)):
        if(dem==0):
            break
        if(inv[day] <= dem):
            dem -= inv[day]
            inv[day] = 0
        else:
            inv[day] -= dem
            dem = 0
    return inv

def updateInventory(inv,item,t,ordered):
    life = shelf_life[item]
    if(item in demand[t].keys()):
        dem = demand[t][item]
    else:
        dem = 0
    # first the orders of the new day are added to the inventory
    # and then the total inventory is used to satisfy the demand
    # This would mean that the demand cannot go unsatisfied from
    # the items in the inventory
    inv = addDay(inv, ordered)
    inv, tu_saved = removeExpired(inv,life,item)
    inv = satisfyFromInventory(inv, dem)
    return inv, tu_saved

def calculateMaxOrderPenalty(schedule):
    max_order_penalty = 0
    for day in schedule.keys():
        for item in schedule[day].keys():
            if(schedule[day][item] > maximum_order_quantity_tu[item]):
                max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
    return max_order_penalty

def calculateEndClearance(inv,item):
    cu_available = sum(inv)
    tu_saved = cu_available//cu_per_tu[item]
    cu_disposed = cu_available%cu_per_tu[item]
    clearance_cost = cu_disposed * clearing_cost_per_cu[item]
    return tu_saved, clearance_cost

def removeFinalOrders(schedule, tu_saved):
    for day in range(len(schedule)-1,-1,-1):
        if(day in schedule.keys()):
            if(tu_saved <= 0):
                break
            if(schedule[day]>=tu_saved):
                schedule[day] -= tu_saved
                break
            else:
                tu_saved -= schedule[day]
                schedule[day] = 0
    return schedule

def neighbourhoodOperator2(schedule):
    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    global new_clearance_cost
    inventory = {}

    for t in time_indexes:
        for item in articles:

            if item not in inventory.keys():
                inventory[item] = []

            # if there was no order for an item in a particular day, then the order was 0
            if(t in schedule.keys() and item in schedule[t].keys()):
                ordered = schedule[t][item] * cu_per_tu[item]
            else:
                ordered = 0

            # update the inventory according to what has been ordered today for item
            inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)
            
            # Removing excess TUs from the schedule. On day t, we remove the excess from day t-shelf life
            if(t-shelf_life[item] in schedule.keys() and item in schedule[t-shelf_life[item]].keys()):
                if(t>=shelf_life[item]):
                    schedule[t-shelf_life[item]][item] -= tu_saved

    schedule = transposeScheduleToItemFirst(schedule)
    for item in schedule.keys():
        tu_saved_at_end, final_clearance_cost = calculateEndClearance(inventory[item],item)
        new_clearance_cost += final_clearance_cost
        schedule[item] = removeFinalOrders(schedule[item], tu_saved_at_end)
    
    schedule = transposeScheduleToDayFirst(schedule)
                
    print("Clearance cost: ", new_clearance_cost)
    print("Ordering cost per TU saved: ", ordering_cost_per_tu_saved)

    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    return schedule, new_clearance_cost, ordering_cost_per_tu_saved, new_max_order_penalty

In [281]:
def transposeDemand(demand):
    result_dict = {}
    for item, days in demand.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][item] = quantity
            else:
                result_dict[day] = {item:quantity}
    return result_dict
demand = transposeDemand(demand)

In [282]:
schedule = transposeScheduleToDayFirst(schedule)
schedule, new_clearance_cost, per_tu_delta, new_max_order_penalty = neighbourhoodOperator2(schedule)
cost_dict['clearance'] = new_clearance_cost
cost_dict['max_order_penalty'] = new_max_order_penalty
cost_dict['per_tu'] -= per_tu_delta
cost_dict['total'] = cost_dict['fixed'] + cost_dict['per_tu'] + cost_dict['buffer'] + cost_dict['clearance'] + cost_dict['max_order_penalty']

Clearance cost:  1789.2000000000003
Ordering cost per TU saved:  0


In [283]:
cost_dict

{'fixed': 7953.5999999999985,
 'per_tu': 60495.79999999998,
 'clearance': 1789.2000000000003,
 'buffer': 149.81307894999975,
 'max_order_penalty': 6.300000000000001,
 'total': 70394.71307894998}

In [ ]:
# max_improvement = math.inf
# cost = obj_val
# modified_schedule = {}
# day_wise_orders = {}

# buffer_penalty_tot = 0
# max_order_penalty_tot = 0
# fixed_ordering_cost_tot = 0 
# schedule_imp = schedule 
# count = 0
# iterator = 0
# MAX_ITER = 100
# MAX_REPEAT = 3
# prev_max_improvement = 0

# while count < MAX_REPEAT and iterator < MAX_ITER:
#     for item in schedule_imp:
#         buffer_penalty = 0
#         max_order_penalty = 0 
#         cost_improvement = 0
#         modified_orders = {}
#         # Check if the item has a second order date
#         if len(schedule_imp[item]) > 1:
#             order_days = list(schedule_imp[item].keys())
#             for i in range (0,len(order_days)-1, 2):
#                 day1 = i
#                 day2 = i + 1
#                 if order_days[day2] - order_days[day1] <= shelf_life[item]:
#                     tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
#                     vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
#                     if vol + volumes[day1] > warehouse_volume:
#                         buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
#                     if tot_orders > maximum_order_quantity_tu[item]:
#                         max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
#                     delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
#                     if delta < 0:
#                         fixed_ordering_cost_tot += ordering_cost_fixed[item] 
#                         buffer_penalty_tot += buffer_penalty
#                         max_order_penalty_tot += max_order_penalty
#                         cost += delta
#                         modified_orders[order_days[day1]] = tot_orders
#                         volumes[order_days[day1]] += vol
#                     else:
#                         modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
#                         modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
#             modified_schedule[item] = modified_orders
        
#         else:
#             if len(schedule[item]) == 1 : 
#                 modified_schedule[item] = schedule_imp[item]
#             else:
#                 continue


#         if cost!=0 and cost < max_improvement:
#             max_improvement = cost
    
#     print("Buffer penalty paid", buffer_penalty_tot)
#     print("max order penalty paid", max_order_penalty_tot)
#     print("fixed cost reduced by", fixed_ordering_cost_tot)
#     schedule_imp = modified_schedule
#     obj_val = max_improvement
#     iterator +=1

#     print(max_improvement)

#     if(max_improvement==prev_max_improvement):
#         count+=1
#     prev_max_improvement = max_improvement